In [11]:
# Union all dataframes and sort
if dfs:
    # Union all dataframes found
    full_df = reduce(DataFrame.unionAll, dfs)
    
    # Sort by created_utc (most recent first)
    # created_utc is likely a float timestamp
    sorted_df = full_df.orderBy(col("created_utc").desc())
    
    # Show result (selecting a few relevant columns)
    display_df = sorted_df.withColumn("created_time", from_unixtime("created_utc")) \
                          .select("subreddit", "created_time", "title", "score", "author")
    
    print(f"Total records: {sorted_df.count()}")
    display_df.show(20, truncate=False)
else:
    print("No data available.")

Total records: 6437


Total records: 6437


Total records: 6437


+---------------+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+--------------------+
|subreddit      |created_time       |title                                                                                                                                                                                                              |score|author              |
+---------------+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+--------------------+
|technology     |2026-01-20 21:13:36|Affordable London to New York flights in under four hours: Boom Supersonic outlines Concorde-esque ambitions                        

In [10]:
# Read Delta tables for each subreddit
dfs = []

for sub in subreddits:
    path = f"s3a://{bucket_name}/{sub}"
    print(f"Attempting to read from: {path}")
    try:
        # Read Delta table
        df = spark.read.format("delta").load(path)
        dfs.append(df)
        print(f"Successfully read data for {sub}. Count: {df.count()}")
    except Exception as e:
        print(f"Error reading {sub} (might not exist yet): {str(e)}")



Attempting to read from: s3a://reddit-streaming-stevenhurwitt-2/technology


26/01/20 21:07:44 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
26/01/20 21:08:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Successfully read data for technology. Count: 1668
Attempting to read from: s3a://reddit-streaming-stevenhurwitt-2/ProgrammerHumor


Successfully read data for ProgrammerHumor. Count: 1724
Attempting to read from: s3a://reddit-streaming-stevenhurwitt-2/news


Successfully read data for news. Count: 1059
Attempting to read from: s3a://reddit-streaming-stevenhurwitt-2/worldnews


Successfully read data for worldnews. Count: 1985


In [9]:
# Initialize Spark Session with Delta and S3 support
# Using the same jars as in the main application
extra_jar_list = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3,org.apache.hadoop:hadoop-common:3.3.1,org.apache.hadoop:hadoop-aws:3.3.1,org.apache.hadoop:hadoop-client:3.3.1,io.delta:delta-spark_2.12:3.2.0,org.postgresql:postgresql:42.5.0"

spark = SparkSession.builder \
    .appName("RedditDeltaRead") \
    .master("local[*]") \
    .config("spark.jars.packages", extra_jar_list) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key", aws_client) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("Spark Session created.")

:: loading settings :: url = jar:file:/opt/venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-edd18444-00cf-4add-b061-d4a76a4402d2;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.3 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in

Spark Session created.


In [6]:
import os
import json
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime
from functools import reduce
from pyspark.sql import DataFrame

# Function to read credentials and config
def find_file(filename, search_paths):
    # Check specific paths first
    for path in search_paths:
        if os.path.exists(path):
            return path
    
    # Walk up from CWD
    curr = os.getcwd()
    while True:
        f = os.path.join(curr, filename)
        if os.path.exists(f):
            return f
        parent = os.path.dirname(curr)
        if parent == curr:
            break
        curr = parent
    return None

def read_config_creds():
    print(f"Current working directory: {os.getcwd()}")
    
    # Creds paths to search
    creds_paths = [
        "creds.json",
        "redditStreaming/creds.json",
        "/home/steven/reddit-streaming/creds.json",
        "/opt/workspace/creds.json",
        "/opt/workspace/redditStreaming/creds.json"
    ]
    
    creds_file = find_file("creds.json", creds_paths)
    if not creds_file:
         raise FileNotFoundError("Could not find creds.json in search paths or parent directories.")
    
    print(f"Found credentials at: {creds_file}")
    with open(creds_file, "r") as f:
        creds = json.load(f)

    # Config paths to search
    config_paths = [
        "config.yaml",
        "redditStreaming/config.yaml",
        "/home/steven/reddit-streaming/config.yaml",
        "/home/steven/reddit-streaming/redditStreaming/config.yaml",
        "/opt/workspace/redditStreaming/config.yaml"
    ]
    
    config_file = find_file("config.yaml", config_paths)
    if not config_file:
         print("Warning: Could not find config.yaml, using defaults or trying alternate locations.")
         # Fallback search if needed, but for now we raise error if strictly required
         # In the original code config was critical for bucket/spark host, but here we need subreddits.
         pass
         
    if config_file:
        print(f"Found config at: {config_file}")
        with open(config_file, "r") as f:
            config = yaml.safe_load(f)
    else:
        # Fallback config if file missing
        config = {"subreddit": ["technology", "ProgrammerHumor", "news", "worldnews"]}
        print("Using default configuration for subreddits.")
        
    return creds, config

creds, config = read_config_creds()

# Extract values
aws_client = creds.get("aws_client")
aws_secret = creds.get("aws_secret")
subreddits = config.get("subreddit", ["technology", "ProgrammerHumor", "news", "worldnews"])
bucket_name = "reddit-streaming-stevenhurwitt-2" 

print(f"Subreddits to process: {subreddits}")

Current working directory: /opt/workspace
Found credentials at: redditStreaming/creds.json
Found config at: redditStreaming/config.yaml
Subreddits to process: ['technology', 'ProgrammerHumor', 'news', 'worldnews']


# Read Delta Tables from S3

This notebook reads reddit data stored in Delta format on S3, unions the data from multiple subreddits, and sorts by most recent.